## BT2101 Group Project 

### Author: Nguyen Ngoc Linh Chi - Matric : A0170767W

In [ ]:
# -------------------------------------------------GOOGLE COLAB UTILITY- ------------------------------------------------------#

# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " 
       + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " 
       + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, 
                                                                                             gpu.memoryUtil*100, 
                                                                                             gpu.memoryTotal))
printm() 

In [ ]:
# -------------------------------------------------MASS IMPORT LIBRARIES ------------------------------------------------------#
# import essential libraries for lading data
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 10000)
import numpy as np

import os
import csv
import math
import time
import pickle
import pandas as pd
import numpy as np
import itertools

import warnings
warnings.filterwarnings('ignore')

# for preprocessing the data
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelBinarizer

from sklearn.decomposition import PCA

# Feature selection
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

# Splitting dataset
from sklearn.model_selection import train_test_split

# Evaluation
from scipy.stats import iqr
from sklearn.metrics import f1_score
from sklearn.metrics import multilabel_confusion_matrix, f1_score, roc_auc_score
from sklearn.model_selection import GridSearchCV, KFold, cross_validate

## Import confusion_matrix, classification_report
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import recall_score, precision_score, f1_score, auc, accuracy_score

from sklearn.base import BaseEstimator, TransformerMixin

# Model
import sklearn
from sklearn import svm
from sklearn import preprocessing

import xgboost as xgb
from xgboost import XGBClassifier

from sklearn import svm
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.kernel_approximation import Nystroem

from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB

from sklearn.linear_model import SGDClassifier, LogisticRegression

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from sklearn.neighbors import KNeighborsClassifier

from sklearn.neural_network import MLPClassifier

from mlxtend.classifier import StackingCVClassifier

# for combining the preprocess with model training
from sklearn.pipeline import make_pipeline, Pipeline, FeatureUnion

# for optimizing the hyperparameters of the pipeline
from sklearn.model_selection import GridSearchCV, KFold, cross_validate, RandomizedSearchCV

# neural network
from keras.models import Sequential
from keras.layers.core import Activation
from keras.layers.core import Dense
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from tensorflow.keras.losses import CategoricalCrossentropy
from itertools import product
from functools import partial
from keras.models import load_model

import keras.backend as K
import csv

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

# Global Settings

In [ ]:
version_code = "_NB_"

# class_weight can be None or Balanced
class_weight_options = ['None', 'Balanced']

# model_type can be Original or Pipeline
models_type_options = ['Original', 'Pipeline']

# model options
model_option = [['None', 'Original']]

# Choose whether you are using colab or your own laptop
setting = 'colab'

# Define paths

In [ ]:
link = 'https://raw.githubusercontent.com/nguyenngoclinhchi/CS3244-Project/master/BT2101Group-Project/card_filter.csv'

def get_paths(mode): 
    if mode == 'colab': 
        # -------------------------------------------------GOOGLE COLAB ROOTPATH ----------------------------------------------#
        ROOT_PATH = '/content/gdrive/My Drive/BT2101_LC_Private'
        ROOT_PATH_MODELS = '/content/gdrive/My Drive/BT2101_LC_Private/models'
        ROOT_PATH_SUBMODELS = '/content/gdrive/My Drive/BT2101_LC_Private/models/submodels'
    else:
        # -------------------------------------------------LAPTOP ROOTPATH ----------------------------------------------------#
        ROOT_PATH = '.'
        ROOT_PATH_MODELS = './models'
        ROOT_PATH_SUBMODELS = './models/submodels'
    return ROOT_PATH, ROOT_PATH_MODELS, ROOT_PATH_SUBMODELS

# Load dataset from Github

In [ ]:
data = pd.read_csv(link, index_col='ID')

# rename last column
data.rename(columns={"default payment next month": "default"}, inplace=True)

X = data.iloc[:,0:23] # input will take from column 1 to 23
y = data.iloc[:,-1] # Last column is output

# X = pd.get_dummies(X)

# Exploratory Data Analysis

In [ ]:
data.shape

In [ ]:
data.info()

In [ ]:
data.describe().T

In [ ]:
%matplotlib inline
%matplotlib notebook

# Plot figures
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl


mpl.rcParams['figure.dpi']= 250
from mpl_toolkits.mplot3d import Axes3D 
plt.style.use('ggplot')

## Correlation analysis

In [ ]:
def heatmap(x, y, size, fig_size):
    fig, ax = plt.subplots(figsize=(fig_size, fig_size))
    
    # Mapping from column names to integer coordinates
    x_labels = [v for v in sorted(x.unique())]
    y_labels = [v for v in sorted(y.unique())]
    x_to_num = {p[1]:p[0] for p in enumerate(x_labels)} 
    y_to_num = {p[1]:p[0] for p in enumerate(y_labels)} 
    
    size_scale = 500
    ax.scatter(
        x=x.map(x_to_num), # Use mapping for x
        y=y.map(y_to_num), # Use mapping for y
        s=size * size_scale, # Vector of square sizes, proportional to size parameter
        marker='s' # Use square as scatterplot marker
    )
    
    # Show column labels on the axes
    ax.set_xticks([x_to_num[v] for v in x_labels])
    ax.set_xticklabels(x_labels, rotation=60, horizontalalignment='right')
    ax.set_yticks([y_to_num[v] for v in y_labels])
    ax.set_yticklabels(y_labels)

corr = X.corr()
corr = pd.melt(corr.reset_index(), id_vars='index')
corr
corr.columns = ['x', 'y', 'value']
heatmap(
    x=corr['x'],
    y=corr['y'],
    size=corr['value'].abs(), fig_size = 15
)

In [ ]:
corr = X.corr()
plt.figure(figsize=(18, 18))
ax = sns.heatmap(
    corr, 
    annot=True,
    square=True
)
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=60,
    horizontalalignment='right'
);

## Distribution plots

In [ ]:
train = X.join(y)
df = train
data_features = df.iloc[:,0:23].columns
print(data_features.__len__())
df.head()

In [ ]:
plt.clf()
plt.figure(figsize=(12,23*4))
gs = mpl.gridspec.GridSpec(23, 1)
for i, cn in enumerate(df[data_features]):
    ax = plt.subplot(gs[i])
    sns.distplot(df[cn][df.default == 1], bins=50)
    sns.distplot(df[cn][df.default == 0], bins=50)
    ax.set_xlabel('')
    ax.set_title('histogram of feature: ' + str(cn))
plt.show()

# Data preprocessing

In [ ]:
def preprocess_input(X):
    # Turn age feature into log(age)
    if ("log10(age+1)" not in X.columns) and ("age" in X.columns):
        X["log10(age+1)"] = X["age"].transform(lambda x: np.log10(x + 1))
        X.drop("age", axis = 1, inplace = True)
    # Drop unnecessary features
    dropped_features = [
        "SEX", "PAY_5", "MARRIAGE", "PAY_3", "EDUCATION", "PAY_2", "PAY_4", "PAY_6"
    ]
    for feature in dropped_features:
        if feature in X.columns:
            X.drop(feature, axis = 1, inplace = True)
    # Turn all category features into multiple one-hot features
    scaler = MinMaxScaler(feature_range=(0,1)).fit(X)
    data_rescaled = scaler.fit_transform(X)
    data_rescaled_df = pd.DataFrame(data_rescaled)
    data_rescaled_df.describe()
    data_rescaled_df.columns = X.columns
    X = data_rescaled_df
    return X

X = preprocess_input(X.copy())

In [ ]:
X_ori = X # save X before using scaler
y_ori = y # save y before using scaler

## Principle Analysis Component

In [ ]:
# Decide what n_components for PCA
## Fitting the PCA algorithm with our Data
pca = PCA().fit(X)

for i in range(np.cumsum(pca.explained_variance_ratio_).__len__()): 
    print("n_component = " + str(i + 1) + " --> " + str(np.cumsum(pca.explained_variance_ratio_)[i]))
#Plotting the Cumulative Summation of the Explained Variance
plt.figure()
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('Number of Components')
plt.ylabel('Variance (%)') #for each component
plt.title('Card Dataset Explained Variance')
plt.show()

In [ ]:
## Based on the above graph --> Choose n = 8
n_component = 8
arr = []
for i in range(n_component): 
    arr.append('#'+ str(i+ 1))
pca = PCA(n_components=n_component)
principalComponents = pca.fit_transform(X.values)

principalDf = pd.DataFrame(data = principalComponents, columns = arr)
finalDf = pd.concat([principalDf, y], axis = 1)
finalDf.head()

# Feature selection

In [ ]:
# Univariate selection 
## apply SelectKBest class to extract features
bestfeatures = SelectKBest(score_func=chi2, k=X.columns.__len__())

fit = bestfeatures.fit(X,y)

dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)

#concat two dataframes for better visualization 
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']  #naming the dataframe columns
print(featureScores.nlargest(X.__len__(),'Score'))

In [ ]:
# Feature importance
from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt
model = ExtraTreesClassifier()

model.fit(X,y)

print(model.feature_importances_, X.columns) #use inbuilt class feature_importances of tree based classifiers

#plot graph of feature importances for better visualization
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(X.__len__()).plot(kind='barh')
plt.figure(figsize=(18, 18))
plt.show()

# Splitting dataset

In [ ]:
#START TRAINING DATA
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 42)

In [ ]:
#START TRAINING DATA
X_train_ori, X_test_ori, y_train_ori, y_test_ori = train_test_split(X_ori, y_ori, test_size = 0.30, random_state = 42)

## Resampling treatement

In [ ]:
def create_meta_data(X_train, y_train): 
    training_data = pd.concat ([X_train,y_train],axis = 1)

    fig = plt.figure(figsize=(5,5)) # Set figsize
    sns.countplot(data=training_data, x='default')
    plt.show()
    ############################################################################################################################
    #                                              RESAMPLING MANUAL                                                           #
    ############################################################################################################################
    # Check default payment next month
    percentage_pay = (data['default'] == 1).sum() / data.shape[0] * 100
    percentage_no_pay = (data['default'] == 0).sum() / data.shape[0] * 100

    print ('default payment next month = 1 : ', percentage_pay)
    print ('default payment next month = 0: ', percentage_no_pay)


    print ('Percentage original pay: ', percentage_pay)
    print ('Percentage original no-pay: ', percentage_no_pay)
    number_of_instances = 30000

    number_sub_pay = int (percentage_pay/100 * number_of_instances)
    number_sub_non_pay = int (percentage_no_pay/100 * number_of_instances)

    sub_pay_data = training_data[training_data['default'] == 1].head(number_sub_pay)
    sub_non_pay_data = training_data[training_data['default'] == 0].head(number_sub_non_pay)

    print ('Number of newly sub pay data:',len(sub_pay_data))
    print ('Number of newly sub non-pay data:',len(sub_non_pay_data))

    sub_training_data = pd.concat ([sub_pay_data, sub_non_pay_data], axis = 0)
    sub_training_data['default'].value_counts()

    ############################################################################################################################
    #                                              UNDER SAMPLING                                                              #
    ############################################################################################################################
    # pay/non-pay data
    # Select row which "default payment next month" is 1 and save in pay_data
    pay_data = sub_training_data[sub_training_data['default'] == 1]
    # Select row which "default payment next month" is 0 and save in non_pay_data
    non_pay_data = sub_training_data[sub_training_data['default'] == 0]

    # Number of pay, non-pay transactions
    number_records_pay = pay_data.shape[0]
    number_records_non_pay = non_pay_data.shape[0]

    # Using sample function on data frame to randomly select number_records_pay from non_pay_data data frame
    under_sample_non_pay = non_pay_data.sample(number_records_pay)
    # **concat** under_sample_non_pay and pay_data to form under_sample_data
    under_sample_data = pd.concat([under_sample_non_pay, pay_data], axis=0)

    under_sample_data.dropna(inplace=True)

    # Showing ratio
    print("Percentage of non pay transactions: ", under_sample_non_pay.shape[0] / under_sample_data.shape[0])
    print("Percentage of pay transactions: ", pay_data.shape[0] / under_sample_data.shape[0])
    print("Total number of transactions in resampled data: ", under_sample_data.shape[0])

    # Assigning X,y for Under-sampled Data
    X_train_undersample = under_sample_data.drop(columns=['default'])
    y_train_undersample = under_sample_data['default']

    # Plot countplot
    plt.figure(figsize=(5,5))
    # Make a count plot to show ratio between 2 default payment next month on "default payment next month" column
    sns.countplot(data=under_sample_data, x='default')
    plt.show()

    ############################################################################################################################
    #                                              OVER SAMPLING                                                               #
    ############################################################################################################################
    # pay/non-pay data
    # Select row which "default payment next month" is 1 and save in pay_data
    pay_data = sub_training_data[sub_training_data['default'] == 1]
    # Select row which "default payment next month" is 0 and save in non_pay_data
    non_pay_data = sub_training_data[sub_training_data['default'] == 0]

    # Number of pay, non-pay transactions
    number_records_pay = pay_data.shape[0]
    number_records_non_pay = non_pay_data.shape[0]

    # Using sample on pay_data with replacement "replace = True",  since we take a larger sample than population
    over_sample_pay = pay_data.sample(replace = True, n=number_records_non_pay)
    # **concat** over_sample_pay and non_pay_data to form under_sample_data
    over_sample_data = pd.concat([over_sample_pay, non_pay_data], axis=0)
    over_sample_data.dropna(inplace=True)

    # Showing ratio
    print("Percentage of non pay transactions: ", non_pay_data.shape[0]/over_sample_data.shape[0])
    print("Percentage of pay transactions: ", pay_data.shape[0]/over_sample_data.shape[0])
    print("Total number of transactions in resampled data: ", over_sample_data.shape[0])

    # Assigning X, y for over-sampled dataset
    X_train_oversample = over_sample_data.drop(columns=['default'])
    y_train_oversample = over_sample_data['default']

    # Plot countplot
    plt.figure(figsize=(5,5))
    # Make a count plot to show ratio between 2 default payment next month on "default payment next month" column
    sns.countplot(data=over_sample_data, x='default')
    plt.show()


    ############################################################################################################################
    #                                              SMOTE                                                                       #
    ############################################################################################################################
    from imblearn.over_sampling import SMOTE
    sm = SMOTE()
    X_smote, y_smote = sm.fit_resample(X_train, y_train)

    ############################################################################################################################
    #                                              ADASYN                                                                      #
    ############################################################################################################################
    from imblearn.over_sampling import ADASYN 
    adasyn = ADASYN()
    X_ada, y_ada = adasyn.fit_resample(X_train, y_train)

    ############################################################################################################################
    #                                              RANDOM UNDER SAMPLER                                                        #
    ############################################################################################################################
    from imblearn.under_sampling import RandomUnderSampler
    rus = RandomUnderSampler()
    X_rus, y_rus = rus.fit_resample(X_train, y_train)

    ############################################################################################################################
    #                                              Combination: SMOTE + ENN                                                    #
    ############################################################################################################################
    from imblearn.combine import SMOTEENN
    smenn = SMOTEENN()
    X_smenn, y_smenn = smenn.fit_resample(X_train, y_train)

    ############################################################################################################################
    #                                              Returns dictionary of different dataset                                     #
    ############################################################################################################################
    meta_data = {
        'Origin': [X_train, y_train],
        'Over sample': [X_train_oversample, y_train_oversample], 
        'Under sample': [X_train_undersample, y_train_undersample], 
        'ADASYN': [X_ada, y_ada],
        'SMOTE': [X_smote, y_smote], 
        'SMOTE + ENN': [X_smenn, y_smenn],
        'Random under sampler': [X_rus, y_rus],
    }
    print("--------------Meta-Data created-------------------")
    return meta_data

# Evaluate and train models

In [ ]:
ROOT_PATH, ROOT_PATH_MODELS, ROOT_PATH_SUBMODELS = get_paths(setting)

In [ ]:
def get_all_models_lists(model_type, class_weight):

    weight_code = 'balanced' if class_weight == 'Balanced' else None

    ############################################################################################################################
    #                                              LinearSVC                                                                   #
    ############################################################################################################################
    scaler = MinMaxScaler()
    model_SVC = LinearSVC(dual=False, multi_class='ovr', max_iter=10000, class_weight = weight_code)
    pipe_SVC = make_pipeline(scaler, model_SVC)

    pipe_steps_SVC = list(pipe_SVC.named_steps.keys())
    print('Pipeline steps:', pipe_steps_SVC)
    model_step_SVC = pipe_steps_SVC[1]

    parameters_SVC = [{
        model_step_SVC + '__' + 'penalty': ['l1', 'l2'], 
        model_step_SVC + '__' + 'loss': ['squared_hinge'],
        model_step_SVC + '__' + 'tol': [0.0001],
        model_step_SVC + '__' + 'C': [1]
    }]
    gscv_SVC = GridSearchCV(
        estimator=pipe_SVC, param_grid=parameters_SVC, scoring="accuracy", cv=5,
        n_jobs=-1, verbose=2, return_train_score=True
    )

    ############################################################################################################################
    #                                              SVM WITH RBF                                                                #
    ############################################################################################################################
    scaler = MinMaxScaler()
    pca = PCA()
    model_SVM = SVC(kernel="rbf", class_weight = weight_code)
    pipe_SVM = make_pipeline(scaler, pca, model_SVM)

    pipe_steps_SVM = list(pipe_SVM.named_steps.keys())
    print('Pipeline steps:', pipe_steps_SVM)
    model_step_PCA = pipe_steps_SVM[1]
    model_step_SVM = pipe_steps_SVM[2]
    parameters_SVM = {
        model_step_PCA + '__n_components': [8],
        model_step_SVM + '__C': [0.1, 0.5, 1, 10, 100, 200],
        model_step_SVM + '__gamma': [0.001, 0.01, 0.1, 1, 10, 50],
    }

    gscv_SVM = GridSearchCV(
        estimator=pipe_SVM, param_grid=parameters_SVM, scoring="accuracy", cv=5,
        n_jobs=-1, verbose=2, return_train_score=True
    )
    
    ############################################################################################################################
    #                                              LogisticRegression                                                          #
    ############################################################################################################################
    scaler = MinMaxScaler()
    model_LR = LogisticRegression(solver="saga", multi_class="ovr", max_iter=10000, tol=0.001, class_weight = weight_code)
    pipe_LR = make_pipeline(scaler, model_LR)

    pipe_steps_LR = list(pipe_LR.named_steps.keys())
    print('Pipeline steps:', pipe_steps_LR)
    model_step_LR = pipe_steps_LR[1]

    parameters_LR = [{
        model_step_LR + '__' + 'penalty': ["l2"],
        model_step_LR + '__' + 'C': [10, 100]
    }]

    gscv_LR = GridSearchCV(
        estimator=pipe_LR, param_grid=parameters_LR, scoring="accuracy", cv=5,
        n_jobs=-1, verbose=2, return_train_score=True
    )


    ############################################################################################################################
    #                                              RF WITH RSCV                                                                #
    ############################################################################################################################
    pipe_RF = make_pipeline(StandardScaler(), RandomForestClassifier(42, class_weight = weight_code))

    pipe_steps_RF = list(pipe_RF.named_steps.keys())
    print('Pipeline steps:', pipe_steps_RF)
    model_step_RF = pipe_steps_RF[1]


    #hyperparameter tuning
    hp_space = {model_step_RF + '__max_depth': np.arange(30,50),
                model_step_RF + '__min_samples_split': np.linspace(0.1, 1.0, 10, endpoint=True),
                model_step_RF + '__min_samples_leaf': np.arange(1,10),
                model_step_RF + '__max_features': list(range(1,len(X_train.columns))),
                model_step_RF + '__n_estimators': [16, 32, 50, 100]
    }
    randcv = RandomizedSearchCV(
        pipe_RF, hp_space, cv=5, verbose=2, scoring='f1_micro', 
        n_iter=80, n_jobs=-1, return_train_score=True
    )

    ############################################################################################################################
    #                                              RF WITH GSCV                                                                #
    ############################################################################################################################
    hp_space2 = {'randomforestclassifier__max_depth': [34, 40],
                'randomforestclassifier__min_samples_split': [0.1, 0.2, 0.3],
                'randomforestclassifier__min_samples_leaf': [3, 4],
                'randomforestclassifier__max_features': [X_train.columns.__len__() - 10, 
                                                         X_train.columns.__len__() - 5, 
                                                         X_train.columns.__len__()],
                'randomforestclassifier__n_estimators': [60, 128]
    }

    gscv2 = GridSearchCV(pipe_RF, hp_space2, cv=5, verbose=2, scoring='f1_micro', 
                         n_jobs=-1, return_train_score=True)

    ############################################################################################################################
    #                                              SGDClassifier                                                               #
    ############################################################################################################################
    scaler = MinMaxScaler()
    model_SGD = SGDClassifier(max_iter=10000, class_weight = weight_code)
    pipe_SGD = make_pipeline(scaler, model_SGD)

    pipe_steps_SGD = list(pipe_SGD.named_steps.keys())
    print('Pipeline steps:', pipe_steps_SGD)
    model_step_SGD = pipe_steps_SGD[1]

    parameters_SGD = [
        {
            model_step_SGD + '__' + 'loss': ['hinge', 'log'],
            model_step_SGD + '__' + 'penalty': ['none', 'l2', 'l1'],
            model_step_SGD + '__' + 'learning_rate': ['optimal'],
            model_step_SGD + '__' + 'alpha': [0.0001]
        },
        {
            model_step_SGD + '__' + 'loss': ['hinge', 'log'],
            model_step_SGD + '__' + 'penalty': ['l2', 'l1'],
            model_step_SGD + '__' + 'learning_rate': ['constant', 'invscaling', 'adaptive'],
            model_step_SGD + '__' + 'alpha': [0.0001],
            model_step_SGD + '__' + 'eta0': [0.1]
        },
        {
            model_step_SGD + '__' + 'loss': ['hinge', 'log'],
            model_step_SGD + '__' + 'penalty': ['none'],
            model_step_SGD + '__' + 'learning_rate': ['constant', 'invscaling', 'adaptive'],
            model_step_SGD + '__' + 'eta0': [0.1]
        }
    ]

    gscv_SGD = GridSearchCV(
        estimator=pipe_SGD, param_grid=parameters_SGD, scoring="accuracy", cv=5,
        n_jobs=-1, verbose=2, return_train_score=True
    )

    ############################################################################################################################
    #                                              XGBClassifier                                                               #
    ############################################################################################################################
    scaler = StandardScaler()
    model_xgb1 = xgb.XGBClassifier(random_state=2000, n_jobs=6, reg_lambda=0, learning_rate=0.2, objective='multi:softmax',
                           max_depth=5, subsample=0.8, colsample_bytree=0.8, seed=100)
    pipe_XGB = make_pipeline(scaler, model_xgb1)

    pipe_steps_XGB = list(pipe_XGB.named_steps.keys())
    print('Pipeline steps:', pipe_steps_XGB)
    model_step_XGB = pipe_steps_XGB[1]

    hp_space1_XGB = {
        model_step_XGB + '__max_depth':range(3,10,2),
        model_step_XGB + '__min_child_weight':range(1,6,2),
        model_step_XGB + '__num_class': [2]
    } 
    gscv_XGB1 = GridSearchCV(
        estimator=pipe_XGB, param_grid=hp_space1_XGB, cv=5, scoring='f1_micro', n_jobs=-1, return_train_score=True, verbose=2
    )

    ############################################################################################################################
    #                                              STACKING                                                                    #
    ############################################################################################################################
    # Initializing Support Vector classifier
    classifier1 = SVC(C = 50, degree = 1, gamma = "auto", kernel = "rbf", probability = True)
    # Initializing Multi-layer perceptron  classifier
    classifier2 = MLPClassifier(activation = "relu", alpha = 0.1, hidden_layer_sizes = (10,10,10),
                                learning_rate = "constant", max_iter = 2000, random_state = 1000)
    # Initialing Nu Support Vector classifier
    classifier3 = NuSVC(degree = 1, kernel = "rbf", nu = 0.25, probability = True)
    # Initializing Random Forest classifier
    classifier4 = RandomForestClassifier(n_estimators = 500, criterion = "gini", max_depth = 10,
                                          max_features = "auto", min_samples_leaf = 0.005,
                                          min_samples_split = 0.005, n_jobs = -1, random_state = 1000)
    # Initializing the StackingCV classifier
    sclf = StackingCVClassifier(classifiers = [classifier1, classifier2, classifier3, classifier4],
                                shuffle = False,
                                use_probas = True,
                                cv = 5,
                                meta_classifier = SVC(probability = True))
    model_step_Stacking = 'meta_classifier'
    params = {
        model_step_Stacking + "__kernel": ["linear", "rbf", "poly"],
        model_step_Stacking + "__C": [1, 2],
        model_step_Stacking + "__degree": [3, 4, 5],
        model_step_Stacking + "__probability": [True]
    }
    gscvStacking1 = GridSearchCV(
        estimator = sclf, 
        param_grid = params, 
        cv = 5, scoring = "roc_auc", verbose = 10, n_jobs = -1)

    ############################################################################################################################
    #                                              Multiple SVM Models                                                         #
    ############################################################################################################################
    scaler = MinMaxScaler()
    pca = PCA()
    model_MultipleSVM = SVC(class_weight = weight_code)
    pipe_MultipleSVM = make_pipeline(scaler, pca, model_MultipleSVM)

    pipe_steps_MultipleSVM = list(pipe_MultipleSVM.named_steps.keys())
    print('Pipeline steps:', pipe_steps_MultipleSVM)
    model_step_PCA = pipe_steps_SVM[1]
    model_step_MultipleSVM = pipe_steps_MultipleSVM[2]
    parameters_MultipleSVM = {
        model_step_PCA + '__n_components': [8],
        model_step_MultipleSVM + "__kernel": ["linear", "rbf", "poly"],
        model_step_MultipleSVM + "__C": [1, 2],
        model_step_MultipleSVM + "__degree": [3, 4, 5],
        model_step_MultipleSVM + "__probability": [True]
    }

    gscv_MultipleSVM = GridSearchCV(
        estimator=pipe_MultipleSVM, param_grid=parameters_SVM, scoring="accuracy", cv=5,
        n_jobs=-1, verbose=2, return_train_score=True
    )

    model_SVM = SVC(kernel="rbf", class_weight = weight_code)
    pipe_SVM = make_pipeline(scaler, pca, model_SVM)

    ############################################################################################################################
    #                                              Simple Models                                                               #
    ############################################################################################################################
    if (model_type == 'Original' and class_weight == 'None'): 
        return {
            # "Logistic Regression"        : [LogisticRegression(),''], 
            # "Decision Tree"              : [DecisionTreeClassifier(),''], 
            # "Random Forest"              : [RandomForestClassifier(),''], 
            # "Bernoulli NB"               : [BernoulliNB(),''], 
            # "Multinomial NB"             : [MultinomialNB(),''], 
            # "Gaussian NB"                : [GaussianNB(),''],
            # "SVM (Linear)"               : [LinearSVC(),''], 
            # "K-Nearest Neighbors"        : [KNeighborsClassifier(),''],
            # "Stochastic Gradient Descent": [SGDClassifier(),'']
        }
    
    if (model_type == 'Original' and class_weight == 'Balanced'): 
        return {
            "Logistic Regression":         [LogisticRegression(class_weight='balanced'),''], 
            "Decision Tree":               [DecisionTreeClassifier(class_weight='balanced'),''], 
            "Random Forest":               [RandomForestClassifier(class_weight='balanced'),''], 
            "SVM (Linear)":                [LinearSVC(class_weight='balanced'),''], 
            "Stochastic Gradient Descent": [SGDClassifier(class_weight='balanced'),'']
        }

    ############################################################################################################################
    #                                              MULTIPLE MODELS                                                             #
    ############################################################################################################################
    if (model_type == 'Pipeline' and class_weight == 'None'): 
        return {
            # 'SVM (Linear)': [gscv_SVC, model_step_SVC],
            # 'Logistic regression': [gscv_LR, model_step_LR],
            # 'Random Forest RandCV': [randcv, model_step_RF],
            # 'Stochastic Gradient Descent': [gscv_SGD, model_step_SGD],
            # 'Random Forest GridCV': [gscv2, model_step_RF], 
            # 'Xtreme Gradient Boosting': [gscv_XGB1, model_step_XGB],
            # 'SVM (Radial Basis Kernel)': [gscv_SVM, model_step_SVM],
            # 'Multiple SVM': [gscv_MultipleSVM, model_step_MultipleSVM],
            'Ensemble model Stacking': [gscvStacking1, model_step_Stacking]
        }
    if (model_type == 'Pipeline') & (class_weight == 'Balanced'):
        return {
            'SVM (Linear)': [gscv_SVC, model_step_SVC],
            'Logistic regression': [gscv_LR, model_step_LR],
            'Random Forest RandCV': [randcv, model_step_RF],
            'Stochastic Gradient Descent': [gscv_SGD, model_step_SGD],
            'Random Forest GridCV': [gscv2, model_step_RF], 
            'SVM (Radial Basis Kernel)': [gscv_SVM, model_step_SVM],
            'Multiple SVM': [gscv_MultipleSVM, model_step_MultipleSVM],
        }

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import recall_score, precision_score, f1_score, auc

# -------------------------------------------------train_models_pipe ----------------------------------------------------------#

def train_models(model, model_step, model_name, model_type, meta_data, class_weight):
    scores_origin = []
    for key, value in meta_data.items(): 
        description = key
        X_train_sub = meta_data[key][0]
        y_train_sub = meta_data[key][1]
        print("----------Dataset: " + key + " with length: " + str(X_train_sub.__len__()))
        try: 
            model.fit(X_train_sub, y_train_sub)
        except: 
            print('Dataset is a DataFrame')
            model.fit(X_train_sub.values, y_train_sub.values)
        # define paths to store values
        ROOT_PATH_MODEL = os.path.join(ROOT_PATH_MODELS, model_name)
        ROOT_PATH_CSV   = ROOT_PATH_MODELS
        model_path      = os.path.join(ROOT_PATH_MODEL, version_code + class_weight + '_' + model_name 
                                      + '.sav')
        model_path_desc = os.path.join(ROOT_PATH_MODEL, version_code + class_weight + '_' + model_name + '_' 
                                      + description + '.sav')
        testing_record_path = os.path.join (ROOT_PATH_CSV, 'Testing Record', 
                                            version_code + class_weight + '_' + model_name + '_' + description  
                                            + '_testing_record.csv')
        training_result_path = os.path.join(ROOT_PATH_CSV, 'Training Record', 
                                            version_code + class_weight + '_' + model_name + '_' + description  
                                            + '_training_record.csv')
        csv_path = os.path.join(ROOT_PATH_CSV, version_code + class_weight + "_" + description + "_" + model_type + 
                                '_' + model_name + '.csv')
        if model_type == 'Original': 
            # Append the current model
            scores_origin.append(evaluate_models(
                model, model_name, model_step, model_path, model_type, testing_record_path, model_path_desc, X_test, y_test, 
                description, class_weight))
        else: 
            # Output the training results
            training_results = pd.DataFrame(model.cv_results_)
            training_results.to_csv(training_result_path, index=False)
            # Append the current model
            scores_origin.append(evaluate_models(
                model, model_name, model_step, model_path, model_type, testing_record_path, 
                model_path_desc, X_test_ori, y_test_ori, 
                description, class_weight))
        print("---------------Updating\n")
        df = pd.DataFrame(scores_origin)
        df.to_csv(csv_path, index=False)
    # always try to update the newest results
    df_origin = pd.DataFrame(scores_origin)
    return df_origin, model, model_name, description

# -------------------------------------------------evaluate_model_pipe --------------------------------------------------------#

def evaluate_models(estimator, model_name, model_step, model_path, model_type, 
                        testing_record_path, model_path_desc, X, y, description, class_weight):
    begin = True
    # Stacking models we use probability to predict
    print("predict values")
    prediction = estimator.predict(X)

    # Set print options
    np.set_printoptions(precision=2)

    # model_name = type(estimator).__name__
    current_score = f1_score(y, prediction, average='micro')

    score = []
    if model_step != '':
        current_model = estimator.best_estimator_
        # save current model
        save_model(model_path_desc, current_model)
        # write down current report model
        test_record_data = {}
        result = {
            'Name': model_name, 
            'F1_score': current_score,
            'Recall': recall_score(y, prediction),
            'Precision': precision_score(y, prediction),
            'Accuracy': accuracy_score(y, prediction),
            'AUC': roc_auc_score(y, prediction),
            'Confusion Matrix': confusion_matrix(y, prediction),
            'Description': description,
            'Remarks': 'Pipeline',
            'Class_weight': class_weight,
            'Current model': estimator.best_estimator_,
            'Best parameters': estimator.best_params_,
            'Best score': estimator.best_score_,
            'Training Time': estimator.refit_time_,
            'Training Score': estimator.best_score_,
        }
        score.append(result)
        score_df = pd.DataFrame(score)
        score_df.to_csv(model_name + "_Result_temp.csv", index=False)
        try: 
            test_record_data = current_model.named_steps[model_step].get_params().copy()
        except: 
            print("Name_steps not in estimator parameters")
        test_record_data["training_time"] = estimator.refit_time_
        test_record_data["training_score"] = estimator.best_score_
        test_record_data["f1_testing_score"] = current_score
        write_down_records(test_record_data, testing_record_path)
        if begin: 
            save_model(model_path, current_model)
            begin = False
        # note the best_model so far
        note_best_model(model_path, current_model, current_score, test_record_data, testing_record_path)
    else: 
        result = {
            'Name': model_name, 
            'F1_score': current_score,
            'Recall': recall_score(y, prediction),
            'Precision': precision_score(y, prediction),
            'Accuracy': accuracy_score(y, prediction),
            'AUC': roc_auc_score(y, prediction),
            'Confusion Matrix': confusion_matrix(y, prediction),
            'Description': description,
            'Remarks': 'Original',
            'Class_weight': class_weight
        }
        score.append(result)
        score_df = pd.DataFrame(score)
        score_df.to_csv(model_name + "_Result_temp.csv", index=False)
    return result

# -------------------------------------------------save_model -----------------------------------------------------------------#

def save_model(model_path, current_model):
    if os.path.isfile(model_path):
        os.remove(model_path)
    pickle.dump(current_model, open(model_path, "wb"))

# -------------------------------------------------write_down_records ---------------------------------------------------------#

def write_down_records(test_record_data, testing_record_path): 
    fieldnames = list(test_record_data.keys())
    with open(testing_record_path, 'w', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerow(test_record_data)
        
# -------------------------------------------------note_best_model ------------------------------------------------------------#

def note_best_model(model_path, current_model, current_score, test_record_data, testing_record_path): 
    best_model = pickle.load(open(model_path, "rb"))
    best_preds = best_model.predict(X_test)
    best_score = f1_score(y_test, best_preds, average='micro')

    if current_score > best_score:
        pickle.dump(current_model, open(model_path, "wb"))
        fieldnames = list(test_record_data.keys())
        with open(testing_record_path, 'a', newline='') as f:
            writer = csv.DictWriter(f, fieldnames=fieldnames)
            writer.writerow(test_record_data)
        best_model = current_model
        best_preds = current_preds
        best_score = current_score

## Train and evaluate models

In [ ]:
trained_classifier = {}
def train_all_models(models_list_pipeline, model_type, class_weight):
    csv_path = os.path.join(ROOT_PATH_MODELS, version_code + class_weight + '_' model_tye + "_" + 'models.csv')
    data_df = {}
    if (model_type == 'Original'):
        data_list = create_meta_data(X_train, y_train)
    else: 
        data_list = create_meta_data(X_train_ori, y_train_ori)
    if class_weight == 'Balanced': # Balanced model needs to train Origin Data only
        data_df['Origin'] = data_list['Origin']
    else: 
        data_df = data_list
    if not os.path.isfile(csv_path):
        meta_model_pipe = pd.DataFrame()
    else: 
        meta_model_pipe = pd.read_csv(csv_path)
    for model_name, model_arr in models_list_pipeline.items():
        print("----Model Name: " + model_name)
        
        # Fit the model
        print("--------Fitting Model !!!")
        model = model_arr[0]
        model_step = model_arr[1]

        # use a copy of meta_data
        model_DF, classifier, model_name, description = train_models(model, model_step, model_name, model_type,
                                                                     data_df, class_weight)
        meta_model_pipe = pd.concat([meta_model_pipe, model_DF], ignore_index=True)
        print("===============PRODUCING CSV FILE FOR MODEL " + model_name + "\n\n")
        meta_model_pipe.sort_values(['F1_score'], ascending=False, inplace=True)
        meta_model_pipe.to_csv(csv_path, index=False)

        trained_classifier[str(model_name + description + class_weight)] = [classifier, description, class_weight]
    return meta_model_pipe

# -------------------------------------------------train_models ---------------------------------------------------------------#
def train_meta_models(model_option): 
    data_df = pd.DataFrame()
    for ele in model_option: 
        class_weight = ele[0]
        model_type = ele[1]
        model_lists = get_all_models_lists(model_type, class_weight)
        data = train_all_models(model_lists, model_type, class_weight)
        data_df = pd.concat([data_df, data], index=False)

    return data_df

In [ ]:
train_meta_models(model_option)